# Testando o modelo do Detector de Emoções com Múltiplas Faces

## Etapa 1 - Importando as bibliotecas

In [1]:
import cv2
import numpy as np
import pandas as pd
#from google.colab.patches import cv2_imshow
import zipfile

In [3]:
%tensorflow_version 2.x

UsageError: Line magic function `%tensorflow_version` not found.


In [2]:
import tensorflow
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array
tensorflow.__version__

'2.3.1'

## Etapa 2 - Conectando com o Drive e acessando os arquivos

In [4]:
from google.colab import drive
drive.mount('/content/gdrive')

ModuleNotFoundError: No module named 'google.colab'

In [ ]:
path = "/content/gdrive/My Drive/Material.zip"
zip_object = zipfile.ZipFile(file = path, mode = "r")
zip_object.extractall('./')
zip_object.close

In [5]:
imagem = cv2.imread('Material/testes/teste03.jpg')
cv2_imshow(imagem)

NameError: name 'cv2_imshow' is not defined

In [ ]:
imagem.shape

## Testando o Detector

### Carregamento dos modelos

In [ ]:
cascade_faces = "Material/haarcascade_frontalface_default.xml"
caminho_modelo = "Material/modelo_01_expressoes.h5"
face_detection = cv2.CascadeClassifier(cascade_faces)
classificador_emocoes = load_model(caminho_modelo, compile = False)
expressoes = ["Raiva", "Nojo", "Medo", "Feliz", "Triste", "Surpreso", "Neutro"]

### Detecção de faces

In [ ]:
faces = face_detection.detectMultiScale(imagem, scaleFactor = 1.2,
                                        minNeighbors = 5, minSize = (20,20))

In [ ]:
faces

In [ ]:
len(faces)

In [ ]:
faces.shape

### Processamento para cada rosto detectado

In [ ]:
cinza = cv2.cvtColor(imagem, cv2.COLOR_BGR2GRAY)
cv2_imshow(cinza)

In [ ]:
cinza.shape

Com o *for* são realizados as etapas para cada face detectada:
* Extração do ROI (region of interest)  
* Redimensionamento
* Normalização
* Previsões e resultado

In [ ]:
original = imagem.copy()

for (x, y, w, h) in faces:
  # Extração do ROI (region of interest)  
  roi = cinza[y:y + h, x:x + w] # utilizamos as coordenadas (onde inicia a face) e a largura e altura para extrair a região de interesse

  # Redimensiona imagem
  roi = cv2.resize(roi, (48, 48))

  cv2_imshow(roi)
  
  # Normalização
  roi = roi.astype("float") / 255
  roi = img_to_array(roi)
  roi = np.expand_dims(roi, axis = 0)

  # Previsões
  preds = classificador_emocoes.predict(roi)[0]
  print(preds)

  # Emoção detectada
  emotion_probability = np.max(preds)
  print(emotion_probability)

  print(preds.argmax())
  label = expressoes[preds.argmax()]


  # Mostra resultado na tela para o rosto
  cv2.putText(original, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.65,
            (0, 0, 255), 2, cv2.LINE_AA)
  cv2.rectangle(original, (x, y), (x + w, y + h), (0, 0, 255), 2)
  


### Exibe resultado final

> Perceba que no resultado final algumas faces não foram detectadas pelo haarscascade. Para solucionar, você pode fazer ajustes nos parâmetros no método `detectMultiScale`

> Na última imagem, o algoritmo detectou duas faces. Também poderiam ser feito ajustes nos parâmetros ou então utilizar o Dlib para a detecção de faces, que é uma biblioteca com resultados melhores

In [ ]:
cv2_imshow(original)